# Acropora Growth Anomaly DNA Analysis

In [1]:
#loading the artifact .qza file generated from the acropora manifest using the command line.
from qiime2 import Artifact, Metadata

single_end_demux_artifact = Artifact.load('single_end_demux.qza')

print(single_end_demux_artifact.uuid)
print(single_end_demux_artifact.type)
print(single_end_demux_artifact.format)

6367d8d1-b155-45ee-8c76-82042131cb61
SampleData[SequencesWithQuality]
<class 'q2_types.per_sample_sequences._format.SingleLanePerSampleSingleEndFastqDirFmt'>


# VSEARCH Method

In [2]:
from qiime2.plugins.vsearch.methods import dereplicate_sequences

#The dereplicated_sequences are the rep_seqs
dereplicated_table, dereplicated_sequences = dereplicate_sequences(sequences = single_end_demux_artifact)

print(dereplicated_sequences)

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: vsearch --derep_fulllength /tmp/q2-QIIME1DemuxDirFmt-mfpkvb_z/seqs.fna --output /tmp/q2-DNAFASTAFormat-5dlqr3op --relabel_sha1 --relabel_keep --uc /tmp/tmprnt7kbtm --xsize --minseqlength 1 --minuniquesize 1 --fasta_width 0



vsearch v2.22.1_linux_x86_64, 7.7GB RAM, 12 cores
https://github.com/torognes/vsearch

Dereplicating file /tmp/q2-QIIME1DemuxDirFmt-mfpkvb_z/seqs.fna 100%
197612659 nt in 135322 seqs, min 152, max 20088, avg 1460
Sorting 100%
135322 unique sequences, avg cluster 1.0, median 1, max 1
Writing FASTA output file 100%
Writing uc file, first part 100%
Writing uc file, second part 100%


<artifact: FeatureData[Sequence] uuid: 36d68dde-e763-4f12-a4ba-3262bdcfcffa>


In [3]:
from qiime2.plugins.vsearch.methods import cluster_features_de_novo

clustered_table, clustered_sequences = cluster_features_de_novo(sequences = dereplicated_sequences, 
                                                               table = dereplicated_table, perc_identity = 0.97)

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: vsearch --cluster_size /tmp/tmpmtwc4gzc --id 0.97 --centroids /tmp/q2-DNAFASTAFormat-uw9b97y_ --uc /tmp/tmpskiv2p6m --qmask none --xsize --threads 1 --minseqlength 1 --fasta_width 0



vsearch v2.22.1_linux_x86_64, 7.7GB RAM, 12 cores
https://github.com/torognes/vsearch

Reading file /tmp/tmpmtwc4gzc 100%
197612659 nt in 135322 seqs, min 152, max 20088, avg 1460
Sorting by abundance 100%
Counting k-mers 100%
Clustering 100%
Sorting clusters 100%
Writing clusters 100%
Clusters: 135322 Size min 1, max 1, avg 1.0
Singletons: 135322, 100.0% of seqs, 100.0% of clusters


In [20]:
print(clustered_sequences)

<artifact: FeatureData[Sequence] uuid: c907c6f6-41db-4e9b-a5df-e74984e44896>


# Taxonomic Analysis (Silva + Mitochondria)

In [4]:
from urllib import request

url = 'https://github.com/zaneveld/GCMP_Global_Disease/raw/master/analysis/organelle_removal/output/silva_metaxa2_reference_taxonomy.qza'
file_name = 'silva_extended_classifier.qza'
request.urlretrieve(url, file_name)

silva_taxonomy = Artifact.load('silva_extended_classifier.qza')
print(silva_taxonomy)

<artifact: FeatureData[Taxonomy] uuid: 36515900-24bf-47bb-9b02-cde7d7a83345>


In [12]:
from Bio import SeqIO
from qiime2 import Artifact
import os

from os import listdir

#Saving the work directory in a variable to make it easier to call.
working_dir = os.path.abspath('/home/ammeix138')

In [13]:
#Using variables to assign the actual path to the files being called. 
#Silva_fasta_path includes the reference sequences 
silva_fasta_path = working_dir + '/Silva_132_release/SILVA_132_QIIME_release/rep_set/rep_set_16S_only/99/silva_132_99_16S.fna'
silva_plus_m2_taxonomy_path = working_dir
m2_silva_otus_path = working_dir + '/m2+silva_otus.fasta'

#Directly importing the data rather than loading the artifact as a .qza
silva_m2_otus = Artifact.import_data('FeatureData[Sequence]', m2_silva_otus_path)

print(silva_m2_otus)


<artifact: FeatureData[Sequence] uuid: ac480fa7-6831-4d26-bd52-5640444c2223>


In [ ]:
#DO NOT RUN THIS BLOCK OF CODE AGAIN UNLESS ABSOLUTELY NECESSARY! IT TAKES UPWARDS OF 5 DAYS TO FINISH RUNNING
#Trying to output a taxonomy artifact using the rep_seqs that were generated from the m2+silva_otus.fasta file
from qiime2.plugins.feature_classifier.pipelines import classify_consensus_vsearch
from urllib import request

taxonomy_silva, = classify_consensus_vsearch(query = clustered_sequences, reference_reads = silva_m2_otus, 
                                             reference_taxonomy = silva_taxonomy)
   

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: vsearch --usearch_global /tmp/qiime2/ammeix138/data/c907c6f6-41db-4e9b-a5df-e74984e44896/data/dna-sequences.fasta --id 0.8 --query_cov 0.8 --strand both --maxaccepts 10 --maxrejects 0 --db /tmp/qiime2/ammeix138/data/ac480fa7-6831-4d26-bd52-5640444c2223/data/dna-sequences.fasta --threads 1 --output_no_hits --blast6out /tmp/q2-BLAST6Format-4wvwonza



vsearch v2.22.1_linux_x86_64, 7.7GB RAM, 12 cores
https://github.com/torognes/vsearch

Reading file /tmp/qiime2/ammeix138/data/ac480fa7-6831-4d26-bd52-5640444c2223/data/dna-sequences.fasta 100%
524471249 nt in 373326 seqs, min 46, max 2961, avg 1405
minseqlength 32: 1 sequence discarded.
Masking 100%
Counting k-mers 100%
Creating k-mer index 100%
Searching

In [ ]:
#Producing an output .qzv file to visuazlize the silva taxonomy summary.
taxonomy_silva_classification = silva_taxonomy.classification

import qiime2.plugins.metadata.actions as metadata_actions

taxonomy_silva_as_md_md = taxonomy_silva_classification.view(Metadata)
taxonomy_silva_viz, = metadata_actions.tabulate(input = taxonomy_silva_as_md_md, )

taxonomy_silva_viz.save('taxonomy_silva_summary.qzv')

In [ ]:
from qiime2.plugins.feature_table.visualizers import summarize

summarize_table, = summarize(table = clustered_table)
summarize_table.save('summary_taxonomy_table.qzv')

In [ ]:
#We generated a sample_metadata file using the original manifest file.
from qiime2 import Metadata

sample_metadata_md = Metadata.load('fastq_manifest_qiime2.txt')

# Bar Plot Visualization Using Silva

In [ ]:
import qiime2.plugins.taxa.actions as taxa_actions

taxa_bar_plots_silva_viz, = taxa_actions.barplot(table = clustered_table, taxonomy = taxonomy_silva_classification,
                                          metadata = sample_metadata_md)
taxa_bar_plots_silva_viz.save('taxa_bar_silva_plots.qzv')

# Taxonomic Analysis

In [15]:
#Generating the classifier to create the taxonomy artifact
from urllib import request

url = 'https://docs.qiime2.org/2021.11/data/tutorials/moving-pictures-usage/gg-13-8-99-515-806-nb-classifier.qza'
fn = 'gg-13-8-99-515-806-nb-classifier.qza'
request.urlretrieve(url, fn)
gg_13_8_99_515_806_nb_classifier = Artifact.load(fn)

#Trying to output a taxonomy artifact using the rep_seqs
import qiime2.plugins.feature_classifier.actions as feature_classifier_actions 

taxonomy = feature_classifier_actions.classify_sklearn(classifier = gg_13_8_99_515_806_nb_classifier,
                                                      reads = dereplicated_sequences)

In [26]:
dir(taxonomy)
taxonomy_classification = taxonomy.classification

In [35]:
import qiime2.plugins.metadata.actions as metadata_actions

taxonomy_as_md_md = taxonomy_classification.view(Metadata)
taxonomy_viz, = metadata_actions.tabulate(
    input=taxonomy_as_md_md,
)

taxonomy_viz.save('taxonomy_summary.qzv')

'taxonomy_summary.qzv'

In [18]:
from qiime2.plugins.feature_table.visualizers import summarize

summarize_table, = summarize(table = clustered_table)
summarize_table.save('summary_taxonomy_table.qzv')

'summary_taxonomy_table.qzv'

In [21]:
#We generated a sample_metadata file using the original manifest file.
from qiime2 import Metadata

sample_metadata_md = Metadata.load('fastq_manifest_qiime2.txt')

# Producing taxa bar plot for visualization

In [142]:
#These commands produce the bar plot visualization that displays the discrepencies of existing microbes
#between healthy and diseased from Acropora cytherea tissue

import qiime2.plugins.taxa.actions as taxa_actions

taxa_bar_plots_viz, = taxa_actions.barplot(table = clustered_table, taxonomy = taxonomy_classification,
                                          metadata = sample_metadata_md)
taxa_bar_plots_viz.save('taxa_bar_plots.qzv')

'taxa_bar_plots.qzv'

# Filtering Sequences Based on Taxa for Mitochondrial DNA

In [144]:
from qiime2.plugins.taxa.methods import filter_seqs

filtered_sequences, = filter_seqs(sequences = dereplicated_sequences, taxonomy = taxonomy_classification,
                                  include = 'mitochondria')
print(filtered_sequences)


<artifact: FeatureData[Sequence] uuid: 24798561-13cc-40df-8ec0-75c4ccde122d>


/home/ammeix138/anaconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/q2_taxa/_method.py:121: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  return sequences[ids_to_keep]
/home/ammeix138/anaconda3/envs/qiime2-2022.11/lib/python3.8/site-packages/q2_types/feature_data/_transformer.py:258: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for id_, seq in data.iteritems():


In [ ]:
from qiime2.plugins.taxa.methods import filter_seqs

unassigned_sequences, = filter_seqs(sequences = dereplicated_sequences, taxonomy = taxonomy_classification,
                                  include = 'Unassigned')
print(unassigned_sequences)

# Filter Table: Taxonomy-based feature table filter

In [145]:
from qiime2.plugins.taxa.methods import filter_table

filter_table_acropora, = filter_table(table = clustered_table, taxonomy = taxonomy_classification, include = 'mitochondria')

print(filter_table_acropora)

<artifact: FeatureTable[Frequency] uuid: 70f21fec-1815-46ec-9d01-a683d27fef09>


# Visualizing The Filtered Sequences and Filter Table

In [146]:
from qiime2.plugins.taxa.visualizers import barplot

filtered_barplot, = barplot(table = filter_table_acropora, taxonomy = taxonomy_classification)
filtered_barplot.save('filtered_bar_plot.qzv')

'filtered_bar_plot.qzv'

# Building Phylogenetic Tree

In [150]:
import qiime2.plugins.phylogeny.actions as phylogeny_actions

action_results = phylogeny_actions.align_to_tree_mafft_fasttree(
    sequences = filtered_sequences)

aligned_reps_seqs = action_results.alignment
masked_aligned_rep_seqs = action_results.masked_alignment
unrooted_tree = action_results.tree
rooted_tree = action_results.rooted_tree

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: mafft --preservecase --inputorder --thread 1 /tmp/qiime2/ammeix138/data/24798561-13cc-40df-8ec0-75c4ccde122d/data/dna-sequences.fasta



inputfile = orig
3 x 20088 - 1667 d
nthread = 1
nthreadpair = 1
nthreadtb = 1
ppenalty_ex = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
    1 / 3 (thread    0)
done.

Constructing a UPGMA tree (efffree=0) ... 
    0 / 3
done.

Progressive alignment 1/2... 
STEP     2 / 2 (thread    0)
done.

Making a distance matrix from msa.. 
    0 / 3 (thread    0)
done.

Constructing a UPGMA tree (efffree=1) ... 
    0 / 3
done.

Progressive alignment 2/2... 
STEP     2 / 2 (thread    0)
done.

disttbfast (nuc) Version 7.508
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
1 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in versio

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: FastTree -quote -nt /tmp/qiime2/ammeix138/data/13fc25f2-1f87-4cf6-96da-09a8ec82b3a3/data/aligned-dna-sequences.fasta



FastTree Version 2.1.11 Double precision (No SSE3)
Alignment: /tmp/qiime2/ammeix138/data/13fc25f2-1f87-4cf6-96da-09a8ec82b3a3/data/aligned-dna-sequences.fasta
Nucleotide distances: Jukes-Cantor Joins: balanced Support: SH-like 1000
Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1
TopHits: 1.00*sqrtN close=default refresh=0.80
ML Model: Jukes-Cantor, CAT approximation with 20 rate categories
Initial topology in 0.00 seconds
Refining topology: 6 rounds ME-NNIs, 2 rounds ME-SPRs, 3 rounds ML-NNIs
Total branch-length 0.644 after 0.00 sec
ML-NNI round 1: LogLk = -30616.845 NNIs 0 max delta 0.00 Time 0.01
Switched to using 20 rate categories (CAT approximation)
Rate categories were divided by 0.633 so that average rate = 1.0
CAT-based log-likelihoods may not be comparable across runs
Use -gamma for approximate but comparable Gamma(20) log-likelihoods
ML-NNI round 2: LogLk = -30469.127 NNIs 0 max delta 0.00 Time 0.02
Turning off heuristics for final round of ML NNIs (converged)

In [156]:
rooted_tree.save('phylogeny_acropora')
unrooted_tree.save('phylogeny_acropora_unrooted')

'phylogeny_acropora_unrooted.qza'